In [2]:
import collections
import os
import random
import requests
import shutil
import zipfile

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.ERROR)

tf.VERSION

'1.2.1'

## Download

In [2]:
HOME_DIR = 'wikipedia'
DATA_DIR = os.path.join(HOME_DIR, 'data')

if not os.path.isdir(DATA_DIR):
    os.makedirs(DATA_DIR)
    
TEXT_URL = 'http://mattmahoney.net/dc/text8.zip'
TEXT_FILENAME = TEXT_URL.split('/')[-1]
TEXT_FILE = os.path.join(DATA_DIR, TEXT_FILENAME)

text_missing = not os.path.isfile(TEXT_FILE)

if text_missing:
    print('Downloading {}...'.format(TEXT_FILENAME))
    r = requests.get(TEXT_URL, stream=True)
    with open(TEXT_FILE, 'wb') as f:
        for chunk in r.iter_content(chunk_size=32768):
            if chunk:
                f.write(chunk)
    print('Done!')

## Vocabulary

In [3]:
def load_raw_text_from_zip(file):
    with zipfile.ZipFile(file) as f:
        return f.read(f.namelist()[0]).decode('utf-8')

raw_text = load_raw_text_from_zip(TEXT_FILE)

print('{}...\n\n({:,d} chars)\n\n...{}'.format(
    raw_text[:1000], len(raw_text) - 2000, raw_text[-1000:]))

 anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a harmonious anti authoritarian society in place of what are regarded as authoritarian political structures and coercive economic instituti

In [4]:
words = raw_text.split()
words_freq = collections.Counter(words).most_common()

print('Words (total):\n\n{:,d}\n'.format(len(words)))
print('Words (unique):\n\n{:,d}\n'.format(len(words_freq)))
print('Most common:\n')
for word, freq in words_freq[:20]:
    print('{} ({:,d})'.format(word, freq))
print('\nLeast common:\n')
for word, freq in words_freq[-20:]:
    print('{} ({:,d})'.format(word, freq))

Words (total):

17,005,207

Words (unique):

253,854

Most common:

the (1,061,396)
of (593,677)
and (416,629)
one (411,764)
in (372,201)
a (325,873)
to (316,376)
zero (264,975)
nine (250,430)
two (192,644)
is (183,153)
as (131,815)
eight (125,285)
for (118,445)
s (116,710)
five (115,789)
three (114,775)
was (112,807)
by (111,831)
that (109,510)

Least common:

jebe (1)
mncs (1)
intitially (1)
privolnoye (1)
gennadi (1)
gorbachyova (1)
democratised (1)
clandenstine (1)
buildups (1)
gorby (1)
kajn (1)
gorbacheva (1)
mikhailgorbachev (1)
englander (1)
workmans (1)
erniest (1)
metzada (1)
metzuda (1)
fretensis (1)
exortation (1)


In [5]:
words_10plus = sum(1 for _, freq in words_freq if freq >= 10)

print('Words 10+: {:,d}'.format(words_10plus))

Words 10+: 47,134


In [6]:
vocabulary_size = 50_000

words_freq[vocabulary_size - 1]

('aggadic', 9)

In [7]:
words_vocab = words_freq[:(vocabulary_size-1)]

print('Words for the vocabulary: {:,d}'.format(len(words_vocab)))

Words for the vocabulary: 49,999


In [8]:
UNK_ID = 0
word_to_id = dict((word, word_id) for word_id, (word, _) in enumerate(words_vocab, UNK_ID+1))
word_to_id['UNK'] = UNK_ID
word_from_id = dict((word_id, word) for word, word_id in word_to_id.items())

print('Vocabulary size: {:d}'.format(len(word_to_id)))

Vocabulary size: 50000


In [9]:
words_to_unk = words_freq[(vocabulary_size-1):]
unk_freq = sum(freq for _, freq in words_to_unk)

print('UNK words: {:,d}'.format(len(words_to_unk)))
print('UNK frequency: {:,d}'.format(unk_freq))

UNK words: 203,855
UNK frequency: 418,391


In [10]:
VOCABULARY_FILE = os.path.join(HOME_DIR, 'vocabulary.txt')

with open(VOCABULARY_FILE, 'w') as f:
    for word_id in range(vocabulary_size):
        f.write(word_from_id[word_id] + '\n')

print('Vocabulary file size: {:,d} bytes'.format(os.stat(VOCABULARY_FILE).st_size))

Vocabulary file size: 418,684 bytes


In [11]:
with open(VOCABULARY_FILE, newline='') as f:
    word_from_id_ = dict((word_id, word.strip()) for word_id, word in enumerate(f))
    word_to_id_ = dict((word, word_id) for word_id, word in word_from_id_.items())

# print(word_from_id_)
print('Vocabulary size: {:,d}'.format(len(word_to_id_)))
assert word_to_id_ == word_to_id
assert word_from_id_ == word_from_id
del word_to_id_, word_from_id_

Vocabulary size: 50,000


In [12]:
data = list(word_to_id.get(word, UNK_ID) for word in words)

print('Size:\n\n{:,d}\n'.format(len(data)))
print('Text (IDs):\n\n{}\n'.format(data[:10]))
print('Text (Words):\n\n{}'.format(list(word_from_id[word_id] for word_id in data[:10])))

Size:

17,005,207

Text (IDs):

[5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156]

Text (Words):

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


## CBOW

**Input**

In [13]:
def context_window(window_words, target_index):
    words = list(window_words)
    del words[target_index]
    return words

def input_cbow(data, batch_size, window_size):
    if window_size % 2 == 0 or window_size < 3 \
        or window_size > (len(data) - batch_size) / 2:
        # {window_size} must be odd: (n words left) target (n words right)
        raise Exception(
            'Invalid parameters: window_size must be a small odd number')

    num_words = len(data)
    num_windows = num_words - window_size + 1
    num_batches = num_windows // batch_size
    target_index = window_size // 2
    
    words = collections.deque(data[window_size:])
    window_words = collections.deque(data[:window_size], maxlen=window_size)
    
    for n in range(num_batches):
        batch = np.ndarray(shape=(batch_size, window_size-1), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

        for i in range(batch_size):
            batch[i,:] = context_window(window_words, target_index)
            labels[i, 0] = window_words[target_index]
            window_words.append(words.popleft())

        yield batch, labels

In [14]:
batch_size = 2
window_size = 3
num_iters = 2
num_words = window_size + num_iters * batch_size - 1
text = ' '.join(word_from_id[word_id] for word_id in data[:num_words])
print('Text\n\n', text, '\n')

data_iter = input_cbow(data, batch_size, window_size)
for k in range(1, num_iters+1):
    print('Batch {}\n'.format(k))
    batch_context, batch_target = next(data_iter)
    for i in range(batch_size):
        context_words = ', '.join(
            word_from_id[word_id] for word_id in batch_context[i, :])
        target_word = word_from_id[batch_target[i, 0]]
        print('[{}] -> {}'.format(context_words, target_word))
    print()

Text

 anarchism originated as a term of 

Batch 1

[anarchism, as] -> originated
[originated, a] -> as

Batch 2

[as, term] -> a
[a, of] -> term



**Model**

In [3]:
graph = tf.Graph()
graph.as_default()
session = tf.InteractiveSession(graph=graph)
session

In [4]:
batch_size = 4
context_size = 2
vocabulary_size = 20
embedding_size = 3
num_sampled = 2

In [5]:
X = tf.constant(np.random.randint(low=0,
                                  high=vocabulary_size,
                                  size=(batch_size, context_size),
                                  dtype=np.int32))

print(X, '\n')
print(X.eval())

Tensor("Const:0", shape=(4, 2), dtype=int32) 

[[ 1 13]
 [12 19]
 [ 2  6]
 [ 6 10]]


In [6]:
y = tf.constant(np.random.randint(low=0,
                                  high=vocabulary_size,
                                  size=(batch_size, 1),
                                  dtype=np.int32))

print(y, '\n')
print(y.eval())

Tensor("Const_1:0", shape=(4, 1), dtype=int32) 

[[12]
 [14]
 [ 4]
 [19]]


In [7]:
# ~ tf.random_uniform(shape=(vocabulary_size, embedding_size),
#                     minval=-1.0, maxval=1.0)
embeddings = tf.Variable(
    2 * np.random.rand(vocabulary_size, embedding_size) - 1, dtype=tf.float32)

embeddings.initializer.run()

print(embeddings, '\n')
print(embeddings.eval())

<tf.Variable 'Variable:0' shape=(20, 3) dtype=float32_ref> 

[[-0.72385198 -0.67064154  0.54728121]
 [ 0.52267903  0.14309864 -0.21202599]
 [ 0.83861756 -0.33369949  0.9575246 ]
 [ 0.96486568  0.56053215  0.49033678]
 [-0.50572079 -0.51958585 -0.73312479]
 [ 0.91628253 -0.02707585  0.9486317 ]
 [-0.06927377 -0.83878195  0.54863465]
 [-0.44515863 -0.57463926  0.65154487]
 [ 0.19133538 -0.53356606  0.61336142]
 [-0.60742682  0.64677942 -0.10579451]
 [-0.90827733  0.05203509 -0.94087756]
 [-0.62333119  0.57111442 -0.38464847]
 [-0.5805648   0.63791192 -0.90240276]
 [-0.72723395  0.40751234 -0.75468791]
 [-0.71756595  0.06584837 -0.73213542]
 [ 0.64148563  0.12542801 -0.12217065]
 [ 0.04314694 -0.74552041  0.77889448]
 [ 0.96567845 -0.91742957  0.55220294]
 [ 0.12808689 -0.78407633  0.60224926]
 [ 0.85479677  0.51144832  0.7075665 ]]


In [8]:
X_embed = tf.nn.embedding_lookup(embeddings, X)

print(X_embed, '\n')
print(X_embed.eval())

Tensor("embedding_lookup:0", shape=(4, 2, 3), dtype=float32) 

[[[ 0.52267903  0.14309864 -0.21202599]
  [-0.72723395  0.40751234 -0.75468791]]

 [[-0.5805648   0.63791192 -0.90240276]
  [ 0.85479677  0.51144832  0.7075665 ]]

 [[ 0.83861756 -0.33369949  0.9575246 ]
  [-0.06927377 -0.83878195  0.54863465]]

 [[-0.06927377 -0.83878195  0.54863465]
  [-0.90827733  0.05203509 -0.94087756]]]


In [9]:
X_avg = tf.reduce_mean(X_embed, axis=1)

print(X_avg, '\n')
print(X_avg.eval())

Tensor("Mean:0", shape=(4, 3), dtype=float32) 

[[-0.10227746  0.27530548 -0.48335695]
 [ 0.13711599  0.57468009 -0.09741813]
 [ 0.3846719  -0.58624071  0.75307965]
 [-0.48877555 -0.39337343 -0.19612145]]


In [10]:
c0_w0 = X_embed[0,:,0].eval()
print('first dimension of each verctor of first context:\n\n', c0_w0, '\n')
print('first dimension avarage:\n\n', np.mean(c0_w0))

first dimension of each verctor of first context:

 [ 0.52267903 -0.72723395] 

first dimension avarage:

 -0.102277


In [11]:
# ~ tf.truncated_normal(shape=(vocabulary_size, embedding_size),
#                       stddev=1.0 / np.sqrt(embedding_size))
W = tf.Variable(
    np.random.randn(vocabulary_size, embedding_size) / np.sqrt(embedding_size),
    dtype=tf.float32)

W.initializer.run()

print(W, '\n')
print(W.eval())

<tf.Variable 'Variable_1:0' shape=(20, 3) dtype=float32_ref> 

[[-0.95111889  0.03992119  0.60304403]
 [-0.25201097  1.22195446  0.91662031]
 [ 0.59678656  1.06454647  0.11580566]
 [-0.52788448 -0.00793689 -0.57282221]
 [ 0.47062469  1.08456814  0.19608852]
 [ 0.80619788  0.15125319  0.05031686]
 [ 0.68311757 -0.86507517 -0.14866897]
 [-0.41752672  0.63992518 -0.928909  ]
 [-0.22810122 -0.79593885  0.03342727]
 [ 0.90696567 -0.15120924 -0.90413463]
 [-0.54319501  0.76286793 -0.88365149]
 [ 1.11157513  0.59892714  1.19862056]
 [ 0.14531991  0.58058619 -0.56140929]
 [ 1.47388983  0.75362307  0.09494124]
 [ 0.52572161 -0.39862293 -0.11135586]
 [-0.42785749 -0.67497361  0.45894542]
 [-0.24799375  0.48624259  0.1878079 ]
 [-0.16138858  0.05955968 -0.83776098]
 [ 0.24626839 -0.42496043  0.37001279]
 [-0.52473778 -0.04243804 -0.46187654]]


In [12]:
# ~ tf.zeros(shape=(vocabulary_size,))
b = tf.Variable(np.zeros(vocabulary_size), dtype=tf.float32)

b.initializer.run()

print(b, '\n')
print(b.eval())

<tf.Variable 'Variable_2:0' shape=(20,) dtype=float32_ref> 

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]


In [13]:
sampled_loss = tf.nn.sampled_softmax_loss(weights=W,
                                          biases=b,
                                          inputs=X_avg,
                                          labels=y,
                                          num_sampled=num_sampled,
                                          num_classes=vocabulary_size)

print(sampled_loss, '\n')
print(sampled_loss.eval())

Tensor("Reshape_2:0", shape=(4,), dtype=float32) 

[ 0.30435109  0.51802754  1.2428782   0.18773946]


In [14]:
loss = tf.reduce_mean(sampled_loss)

print(loss, '\n')
print(loss.eval())

Tensor("Mean_1:0", shape=(), dtype=float32) 

0.683698


In [15]:
session.close()
del X, y, embeddings, X_embed, X_avg, c0_w0, W, b, sampled_loss, loss
del graph, session

In [28]:
def model_cbow(vocabulary_size, embedding_size, num_sampled):
    X = tf.placeholder_with_default([[0]], shape=(None, None), name='X')
    y = tf.placeholder_with_default([[0]], shape=(None, 1), name='y')
    
    embeddings = tf.Variable(
        tf.random_uniform(shape=(vocabulary_size, embedding_size),
                          minval=-1.0, maxval=1.0),
        name='embeddings')

    X_embed = tf.nn.embedding_lookup(embeddings, X)
    X_avg = tf.reduce_mean(X_embed, axis=1)
    
    softmax_weights = tf.Variable(
        tf.truncated_normal(shape=(vocabulary_size, embedding_size),
                            stddev=1.0 / np.sqrt(embedding_size)),
        name='W')
    softmax_biases = tf.Variable(
        tf.zeros(shape=(vocabulary_size,)),
        name='b')
    
    with tf.name_scope('loss'):
        sampled_loss = tf.nn.sampled_softmax_loss(weights=softmax_weights,
                                                  biases=softmax_biases,
                                                  inputs=X_avg,
                                                  labels=y,
                                                  num_sampled=num_sampled,
                                                  num_classes=vocabulary_size)
        loss = tf.reduce_mean(sampled_loss, name='mean')

    norm = tf.norm(embeddings, axis=1, keep_dims=True)
    normalized_embeddings = embeddings / norm

    return X, y, normalized_embeddings, loss

In [29]:
batch_size = 4
context_size = 2
vocabulary_size = 20
embedding_size = 3
num_sampled = 2

with tf.Graph().as_default() as graph, \
    tf.Session(graph=graph) as session:

    X, y, embeddings, loss_op = model_cbow(vocabulary_size,
                                           embedding_size,
                                           num_sampled)

    tf.global_variables_initializer().run()

    X_batch = np.random.randint(low=0,
                                high=vocabulary_size,
                                size=(batch_size, context_size),
                                dtype=np.int32)
    y_batch = np.random.randint(low=0,
                                high=vocabulary_size,
                                size=(batch_size, 1),
                                dtype=np.int32)
    feed_data = {X: X_batch, y: y_batch}

    loss, embeddings_ = session.run([loss_op, embeddings], feed_dict=feed_data)

    print('Avarage loss: {:,.3f}\n'.format(loss))
    print(embeddings_)

Avarage loss: 0.659

[[-0.35204613 -0.49285588 -0.79571134]
 [ 0.90027946  0.00527231 -0.43528083]
 [-0.45081681 -0.8457576   0.28540915]
 [ 0.93553668 -0.06053047  0.34800476]
 [ 0.62018192  0.77692211 -0.10847333]
 [-0.05211216  0.72057694  0.691414  ]
 [ 0.85853469 -0.40509552 -0.31434992]
 [ 0.65642017 -0.38012969  0.65162414]
 [ 0.25981882 -0.48941576  0.8324461 ]
 [ 0.02589937 -0.97443479 -0.22317269]
 [-0.81627828  0.2518996  -0.51984262]
 [ 0.69813287  0.71491545  0.03881122]
 [-0.76648676  0.62046701  0.16588779]
 [-0.03354356  0.7318964  -0.6805898 ]
 [-0.8009752   0.15727918 -0.5776695 ]
 [ 0.07402308 -0.83239329  0.54921955]
 [ 0.16653079 -0.84062201  0.51538551]
 [-0.01632886 -0.62661898  0.77915466]
 [ 0.73106146 -0.66975462  0.13029923]
 [ 0.16645701  0.67231679  0.72130591]]


## Skip-gram Model

In [30]:
def context_window(window_words, target_index):
    words = list(window_words)
    del words[target_index]
    return words

def context_sample(context_words, sample_size):
    return random.sample(context_words, sample_size)

def context_skips(window_words, target_index, sample_size, use_sample):
    words = context_window(window_words, target_index)
    if use_sample:
        words = context_sample(words, sample_size) 
    return words

def input_skip_gram(data, batch_size, window_size, num_skips):
    if window_size % 2 == 0 or window_size < 3 \
        or window_size > (len(data) - batch_size) / 2:
        # {window_size} must be odd: (n words left) target (n words right)
        raise Exception(
            'Invalid parameters: window_size must be a small odd number')
    if num_skips > window_size - 1:
        # It is not possible to generate {num_skips} different pairs
        # with the second word coming from {window_size - 1} words.
        raise Exception(
            'Invalid parameters: num_skips={}, window_size={}'.format(
                num_skips, window_size))

    num_words = len(data)
    num_windows = num_words - window_size + 1
    num_batches = num_windows * num_skips // batch_size
    target_index = window_size // 2
    use_sample = num_skips < window_size - 1

    words = collections.deque(data[window_size:])
    window_words = collections.deque(data[:window_size], maxlen=window_size)
    target_word = window_words[target_index]
    context_words = context_skips(window_words,
                                  target_index,
                                  num_skips,
                                  use_sample)

    for n in range(num_batches):
        batch = np.ndarray(shape=(batch_size,), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

        for i in range(batch_size):
            batch[i] = target_word
            labels[i, 0] = context_words.pop()
            if not context_words:
                window_words.append(words.popleft())
                target_word = window_words[target_index]
                context_words = context_skips(window_words,
                                              target_index,
                                              num_skips,
                                              use_sample)

        yield batch, labels

In [31]:
batch_size = 2
window_size = 3
num_skips = 2
num_iters = 2
num_words = window_size + num_iters * batch_size // num_skips - 1
text = ' '.join(word_from_id[word_id] for word_id in data[:num_words])
print('Text\n\n', text, '\n')

data_iter = input_skip_gram(data, batch_size, window_size, num_skips)
for k in range(1, num_iters+1):
    print('Batch {}\n'.format(k))
    batch_target, batch_context = next(data_iter)
    for i in range(batch_size):
        target_word = word_from_id[batch_target[i]]
        context_word = word_from_id[batch_context[i, 0]]
        print('{} -> {}'.format(target_word, context_word))
    print()


Text

 anarchism originated as a 

Batch 1

originated -> as
originated -> anarchism

Batch 2

as -> a
as -> originated



**Model**

In [16]:
graph = tf.Graph()
graph.as_default()
session = tf.InteractiveSession(graph=graph)
session

In [17]:
batch_size = 4
vocabulary_size = 20
embedding_size = 3
num_sampled = 2

In [18]:
X = tf.constant(np.random.randint(low=0,
                                  high=vocabulary_size,
                                  size=(batch_size,),
                                  dtype=np.int32))

print(X, '\n')
print(X.eval())

Tensor("Const:0", shape=(4,), dtype=int32) 

[ 1 13  7 14]


In [19]:
y = tf.constant(np.random.randint(low=0,
                                  high=vocabulary_size,
                                  size=(batch_size, 1),
                                  dtype=np.int32))

print(y, '\n')
print(y.eval())

Tensor("Const_1:0", shape=(4, 1), dtype=int32) 

[[5]
 [8]
 [4]
 [2]]


In [20]:
# ~ tf.random_uniform(shape=(vocabulary_size, embedding_size),
#                     minval=-1.0, maxval=1.0)
embeddings = tf.Variable(
    2 * np.random.rand(vocabulary_size, embedding_size) - 1, dtype=tf.float32)

embeddings.initializer.run()

print(embeddings, '\n')
print(embeddings.eval())

<tf.Variable 'Variable:0' shape=(20, 3) dtype=float32_ref> 

[[-0.06405666 -0.36442509  0.09976698]
 [ 0.25951034 -0.92962027  0.29985774]
 [-0.07028455  0.21121742  0.75075257]
 [-0.0650936  -0.7252804   0.72301978]
 [-0.32284504  0.03299018 -0.09381296]
 [-0.39663857 -0.24010691 -0.71467304]
 [-0.40799075  0.31186756  0.7314164 ]
 [ 0.47215068 -0.03772468 -0.52421725]
 [ 0.72830141  0.5357722  -0.0377    ]
 [-0.81130475  0.77849263  0.3450298 ]
 [-0.69996762 -0.48169607  0.65136951]
 [ 0.05303586 -0.98581433  0.8968181 ]
 [ 0.72462726  0.97945833  0.76254189]
 [-0.25559229 -0.44913685 -0.3040261 ]
 [ 0.94620019  0.64193922  0.59860039]
 [ 0.72233653  0.12625705 -0.33893391]
 [ 0.84457082  0.03674772 -0.04018029]
 [ 0.24594459 -0.20061782 -0.84947133]
 [ 0.81655848 -0.19471201  0.66972214]
 [ 0.12862396 -0.88693208  0.59677744]]


In [21]:
X_embed = tf.nn.embedding_lookup(embeddings, X)

print(X_embed, '\n')
print(X_embed.eval())

Tensor("embedding_lookup:0", shape=(4, 3), dtype=float32) 

[[ 0.25951034 -0.92962027  0.29985774]
 [-0.25559229 -0.44913685 -0.3040261 ]
 [ 0.47215068 -0.03772468 -0.52421725]
 [ 0.94620019  0.64193922  0.59860039]]


In [22]:
# ~ tf.truncated_normal(shape=(vocabulary_size, embedding_size),
#                       stddev=1.0 / np.sqrt(embedding_size))
W = tf.Variable(
    np.random.randn(vocabulary_size, embedding_size) / np.sqrt(embedding_size),
    dtype=tf.float32)

W.initializer.run()

print(W, '\n')
print(W.eval())

<tf.Variable 'Variable_1:0' shape=(20, 3) dtype=float32_ref> 

[[ 1.79186618  0.24223483  0.07708248]
 [-0.88965774 -0.37394619 -0.14469676]
 [ 0.2534678  -0.13020152 -0.27742332]
 [-0.05867928 -0.02351343  0.37994054]
 [ 0.47904381 -0.90131861 -0.06806456]
 [-0.47352576  0.84705657 -0.39652976]
 [-0.38010231 -0.75899756 -0.37158328]
 [ 0.28823701 -0.59937847 -1.06404757]
 [ 0.1208643   0.36876652  0.44599587]
 [ 0.32415268 -0.21567565 -0.09200609]
 [-0.04196546  0.42018735 -0.81791347]
 [-0.11105991 -0.78590721 -0.36691234]
 [-1.52351332 -0.98226297 -0.14049503]
 [-0.69547641 -1.20471764  0.14849003]
 [ 0.43098313 -0.44809586 -0.36235431]
 [-0.44773129 -1.59782124  0.34702504]
 [-0.57949483 -0.18733098  1.04739034]
 [-0.07380351 -0.61254019  0.40654027]
 [ 0.15701263  0.83129191  0.49949175]
 [ 0.20048323 -0.60173309 -0.37230048]]


In [23]:
# ~ tf.zeros(shape=(vocabulary_size,))
b = tf.Variable(np.zeros(vocabulary_size), dtype=tf.float32)

b.initializer.run()

print(b, '\n')
print(b.eval())

<tf.Variable 'Variable_2:0' shape=(20,) dtype=float32_ref> 

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]


In [24]:
sampled_loss = tf.nn.sampled_softmax_loss(weights=W,
                                          biases=b,
                                          inputs=X_embed,
                                          labels=y,
                                          num_sampled=num_sampled,
                                          num_classes=vocabulary_size)

print(sampled_loss, '\n')
print(sampled_loss.eval())

Tensor("Reshape_2:0", shape=(4,), dtype=float32) 

[ 1.2450664   0.51244265  0.71858579  1.3334657 ]


In [25]:
loss = tf.reduce_mean(sampled_loss)

print(loss, '\n')
print(loss.eval())

Tensor("Mean:0", shape=(), dtype=float32) 

1.06209


In [26]:
session.close()
del X, y, embeddings, X_embed, W, b, sampled_loss, loss
del graph, session

In [43]:
def model_skip_gram(vocabulary_size, embedding_size, num_sampled):
    X = tf.placeholder_with_default([0], shape=(None,), name='X')
    y = tf.placeholder_with_default([[0]], shape=(None, 1), name='y')
    
    embeddings = tf.Variable(
        tf.random_uniform(shape=(vocabulary_size, embedding_size),
                          minval=-1.0, maxval=1.0),
        name='embeddings')

    X_embed = tf.nn.embedding_lookup(embeddings, X)

    softmax_weights = tf.Variable(
        tf.truncated_normal(shape=(vocabulary_size, embedding_size),
                            stddev=1.0 / np.sqrt(embedding_size)),
        name='W')
    softmax_biases = tf.Variable(
        tf.zeros(shape=(vocabulary_size,)),
        name='b')

    with tf.name_scope('loss'):
        sampled_loss = tf.nn.sampled_softmax_loss(weights=softmax_weights,
                                                  biases=softmax_biases,
                                                  inputs=X_embed,
                                                  labels=y,
                                                  num_sampled=num_sampled,
                                                  num_classes=vocabulary_size)
        loss = tf.reduce_mean(sampled_loss, name='mean')

    norm = tf.norm(embeddings, axis=1, keep_dims=True)
    normalized_embeddings = embeddings / norm

    return X, y, normalized_embeddings, loss

In [44]:
batch_size = 4
vocabulary_size = 20
embedding_size = 3
num_sampled = 2

with tf.Graph().as_default() as graph, \
    tf.Session(graph=graph) as session:

    X, y, embeddings, loss_op = model_skip_gram(vocabulary_size,
                                                embedding_size,
                                                num_sampled)

    tf.global_variables_initializer().run()

    X_batch = np.random.randint(low=0,
                                high=vocabulary_size,
                                size=(batch_size,),
                                dtype=np.int32)
    y_batch = np.random.randint(low=0,
                                high=vocabulary_size,
                                size=(batch_size, 1),
                                dtype=np.int32)
    feed_data = {X: X_batch, y: y_batch}

    loss, embeddings_ = session.run([loss_op, embeddings], feed_dict=feed_data)

    print('Avarage loss: {:,.3f}\n'.format(loss))
    print(embeddings_)

Avarage loss: 0.848

[[ 0.36134785 -0.71822989  0.59462047]
 [ 0.3036077   0.49467725 -0.81431985]
 [-0.24205504  0.0727653   0.96753013]
 [ 0.49863282  0.29093915 -0.8165291 ]
 [ 0.87326252 -0.36429736 -0.32357374]
 [ 0.47323585 -0.74552345  0.46930027]
 [-0.52143198  0.1009413   0.8473013 ]
 [-0.24573667  0.50146824  0.82954383]
 [-0.43916667 -0.76209062  0.47576305]
 [ 0.47012448  0.49093866  0.73345912]
 [ 0.51530111 -0.67711431  0.52533883]
 [ 0.87959242  0.45998335 -0.12137818]
 [ 0.45687237 -0.80557758 -0.37724325]
 [-0.073541    0.7254191   0.68436748]
 [-0.59740579 -0.767735   -0.23170929]
 [ 0.78319734  0.52396584 -0.3347564 ]
 [ 0.85303044 -0.51144564  0.10374222]
 [-0.52321196 -0.6087727  -0.59635991]
 [ 0.58028489 -0.66106236 -0.47567412]
 [-0.90289181 -0.34156856 -0.260993  ]]


## Nearest Neighbors

In [45]:
graph = tf.Graph()
graph.as_default()
session = tf.InteractiveSession(graph=graph)
session

In [46]:
v_0 = tf.constant([3, 4], dtype=tf.float32)
v_1 = tf.constant([4, 3], dtype=tf.float32)
v_2 = tf.constant([-3, 4], dtype=tf.float32)
v_3 = tf.constant([-4, 3], dtype=tf.float32)

V = tf.stack([v_0, v_1, v_2, v_3])

print(V, '\n')
print(V.eval())

Tensor("stack:0", shape=(4, 2), dtype=float32) 

[[ 3.  4.]
 [ 4.  3.]
 [-3.  4.]
 [-4.  3.]]


In [47]:
V_norm = tf.norm(V, axis=1, keep_dims=True)

print(V_norm, '\n')
print(V_norm.eval())

Tensor("norm/Sqrt:0", shape=(4, 1), dtype=float32) 

[[ 5.]
 [ 5.]
 [ 5.]
 [ 5.]]


In [48]:
U = V / V_norm

print(U, '\n')
print(U.eval())

Tensor("truediv:0", shape=(4, 2), dtype=float32) 

[[ 0.60000002  0.80000001]
 [ 0.80000001  0.60000002]
 [-0.60000002  0.80000001]
 [-0.80000001  0.60000002]]


In [49]:
UU = tf.diag_part(tf.matmul(U, U, transpose_b=True))

print(UU, '\n')
print(UU.eval())

Tensor("DiagPart:0", shape=(4,), dtype=float32) 

[ 1.  1.  1.  1.]


In [50]:
i = tf.constant([0], dtype=tf.int32)

u_i = tf.nn.embedding_lookup(U, i)

print(u_i, '\n')
print(u_i.eval())

Tensor("embedding_lookup:0", shape=(1, 2), dtype=float32) 

[[ 0.60000002  0.80000001]]


In [51]:
S = tf.matmul(u_i, U, transpose_b=True)

print(S, '\n')
print(S.eval())

Tensor("MatMul_1:0", shape=(1, 4), dtype=float32) 

[[  1.00000000e+00   9.60000038e-01   2.80000001e-01   7.15255766e-09]]


In [52]:
nn_values, nn_indices = tf.nn.top_k(S, 2)

print(nn_values, '\n')
print(nn_values.eval(), '\n')
print(nn_indices, '\n')
print(nn_indices.eval())

Tensor("TopKV2:0", shape=(1, 2), dtype=float32) 

[[ 1.          0.96000004]] 

Tensor("TopKV2:1", shape=(1, 2), dtype=int32) 

[[0 1]]


In [53]:
session.close()
del v_0, v_1, v_2, v_3, V, V_norm, U
del i, u_i, S, nn_values, nn_indices
del graph, session

In [54]:
class NearestWordsQuery:
    
    def __init__(self, word_from_id, words, k=4):
        self.word_from_id = word_from_id
        self.words = words
        self.k = k

    def build_graph(self, embeddings, name=None):
        with tf.name_scope(name, "nearest_words", [self.words, self.k]):
            input_words = tf.placeholder(tf.int32, shape=(None,))

            input_embed = tf.nn.embedding_lookup(embeddings, input_words)
            similarity = tf.matmul(input_embed, embeddings, transpose_b=True)
            nearest = tf.nn.top_k(similarity, self.k+1)

        self.input_words = {input_words: self.words}
        self.nearest = nearest
    
    def nearest_words(self, target_id, nearest_indices, nearest_values):
        id_pairs = zip(nearest_indices, nearest_values)
        word_pairs = list((self.word_from_id[word_id], value)
                          for word_id, value in id_pairs
                          if word_id != target_id)
        return word_pairs[:self.k]
    
    def format_words(self, word_pairs):
        return ('{} ({:,.3f})'.format(word, value)
                for word, value in word_pairs)
    
    def run(self, session):
        nearest_val, nearest_id = session.run(self.nearest,
                                              feed_dict=self.input_words)
        for i, word_id in enumerate(self.words):
            word = self.word_from_id[word_id]
            nearest_words = self.nearest_words(
                word_id, nearest_id[i], nearest_val[i])
            nearest_words = ', '.join(self.format_words(nearest_words))
            print('{}: {}'.format(word, nearest_words))

In [55]:
rev_vocab = {0: 'unk', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e'}
vocabulary_size = len(rev_vocab)
embedding_size = 3

nn = NearestWordsQuery(rev_vocab, words=[2, 5], k=2)

with tf.Graph().as_default() as graph, \
    tf.Session(graph=graph) as session:
    
    V = 2 * np.random.rand(vocabulary_size, embedding_size) - 1
    U = V / np.linalg.norm(V, axis=1, keepdims=True)
    embeddings = tf.constant(U)
    
    nn.build_graph(embeddings)
    nn.run(session)

b: e (0.840), unk (0.616)
e: b (0.840), c (0.419)


## Experiments

In [ ]:
def opt_adagrad(loss, learning_rate=1.0):
    return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.train.get_or_create_global_step(),
        learning_rate=learning_rate,
        optimizer='Adagrad')

In [ ]:
def train(model_fn, input_fn, opt_fn, query,
          num_epochs=1, model_dir='/tmp/embedding_model', remove_model=True):
    if remove_model and os.path.isdir(model_dir):
        shutil.rmtree(model_dir)

    with tf.Graph().as_default():
        X, y, embeddings, loss_op = model_fn()
        train_op = opt_fn(loss_op)

        query.build_graph(embeddings)

        with tf.train.MonitoredTrainingSession(
            checkpoint_dir=model_dir) as session:
            
            for epoch in range(1, num_epochs+1):
                print('Epoch {}\n'.format(epoch))

                avg_loss = 0
                for step, (X_batch, y_batch) in enumerate(input_fn()):
                    _, loss = session.run([train_op, loss_op],
                                          feed_dict={X: X_batch, y: y_batch})

                    avg_loss = (loss + step * avg_loss) / (step + 1)
                    if step % 10_000 == 0:
                        print('...{:,d} Avarage loss: {:.3f}'.format(
                            step, avg_loss))

                print('\nAvarage loss: {:.3f}\n'.format(avg_loss))
                query.run(session)
                print()

            return session.run(embeddings)

In [68]:
def save_embeddings(file, embeddings):
    with open(file, 'w') as f:
        vocabulary_size = embeddings.shape[0] 
        for word_id in range(vocabulary_size):
            embedding = embeddings[word_id]
            embedding_string = ('{:.5f}'.format(k) for k in embedding)
            embedding_string = ' '.join(embedding_string)
            f.write(embedding_string)
            f.write('\n')

In [57]:
valid_num_words = 8
valid_range_words = 1000
valid_words = random.sample(range(1, valid_range_words), valid_num_words)

for word_id in valid_words:
    print(word_from_id[word_id])

each
length
writer
great
go
literature
seven
examples


In [58]:
nearest_words = NearestWordsQuery(word_from_id, valid_words, 4)

In [59]:
%%time

MODEL_DIR = os.path.join(HOME_DIR, 'cbow')

vocabulary_size = len(word_to_id)
embedding_size = 128
num_sampled = 64

batch_size = 128
window_size = 3

model_fn = lambda: model_cbow(vocabulary_size, embedding_size, num_sampled)
input_fn = lambda: input_cbow(data, batch_size, window_size)
opt_fn = lambda loss: opt_adagrad(loss, learning_rate=1.0)

cbow_embeddings = train(model_fn,
                        input_fn,
                        opt_fn,
                        nearest_words,
                        num_epochs=1,
                        model_dir=MODEL_DIR)

Epoch 1

...0 Avarage loss: 7.609
...10,000 Avarage loss: 3.445
...20,000 Avarage loss: 3.273
...30,000 Avarage loss: 3.186
...40,000 Avarage loss: 3.118
...50,000 Avarage loss: 3.076
...60,000 Avarage loss: 3.035
...70,000 Avarage loss: 2.999
...80,000 Avarage loss: 2.969
...90,000 Avarage loss: 2.944
...100,000 Avarage loss: 2.916
...110,000 Avarage loss: 2.886
...120,000 Avarage loss: 2.869
...130,000 Avarage loss: 2.847

Avarage loss: 2.844

each: every (0.632), any (0.551), all (0.357), incitement (0.327)
length: variation (0.377), maximum (0.375), halting (0.363), speed (0.360)
writer: author (0.525), politician (0.521), mathematician (0.519), poet (0.511)
great: little (0.422), dearborn (0.397), soi (0.373), considerable (0.365)
go: went (0.380), move (0.362), pass (0.358), preventative (0.351)
literature: texts (0.400), beaverbrook (0.357), markup (0.344), playwright (0.340)
seven: eight (0.869), five (0.833), six (0.832), four (0.826)
examples: aspects (0.431), elements (0.409

In [65]:
cbow_embeddings[0]

array([ 0.01355005, -0.19189504,  0.08304871,  0.14788906, -0.01292881,
        0.08955733,  0.06460255, -0.01952947,  0.10569677,  0.01884526,
       -0.01156916, -0.14599091,  0.05206586,  0.10930529,  0.11521848,
       -0.03579207, -0.17535175,  0.13498679, -0.11360314,  0.00087663,
       -0.00106169, -0.05076494,  0.13074888,  0.00617049, -0.0657478 ,
        0.03324445, -0.09406804,  0.13334005,  0.03737927,  0.03893398,
        0.0273369 , -0.09000934, -0.02187724,  0.10807586, -0.07821658,
        0.02211384, -0.13984069, -0.07788607,  0.02657686,  0.07809026,
        0.01229459,  0.03779913, -0.05638366,  0.04662901, -0.08944251,
        0.07706796, -0.04529163,  0.02438457, -0.09905559, -0.06779324,
        0.04311862,  0.08599722,  0.01276149, -0.04310194,  0.02160779,
       -0.08086037,  0.06877899,  0.02897387,  0.04288683, -0.04100583,
       -0.02869713,  0.08337607,  0.03640424, -0.07170308, -0.03564997,
       -0.00045511, -0.01639324,  0.10533239,  0.0118693 ,  0.07

In [70]:
EMBEDDINGS_FILE = os.path.join(HOME_DIR, 'cbow.txt')
save_embeddings(EMBEDDINGS_FILE, cbow_embeddings)

print('Embeddings file size: {:,d} bytes'.format(os.stat(EMBEDDINGS_FILE).st_size))

Embeddings file size: 54,399,864 bytes


In [60]:
%%time

MODEL_DIR = os.path.join(HOME_DIR, 'skip_gram')

vocabulary_size = len(word_to_id)
embedding_size = 128
num_sampled = 64

batch_size = 128
window_size = 3
num_skips = 2

model_fn = lambda: model_skip_gram(vocabulary_size, embedding_size, num_sampled)
input_fn = lambda: input_skip_gram(data, batch_size, window_size, num_skips)
opt_fn = lambda loss: opt_adagrad(loss, learning_rate=1.0)

skip_embeddings = train(model_fn,
                        input_fn,
                        opt_fn,
                        nearest_words,
                        num_epochs=1,
                        model_dir=MODEL_DIR)

Epoch 1

...0 Avarage loss: 8.529
...10,000 Avarage loss: 3.854
...20,000 Avarage loss: 3.685
...30,000 Avarage loss: 3.615
...40,000 Avarage loss: 3.570
...50,000 Avarage loss: 3.541
...60,000 Avarage loss: 3.517
...70,000 Avarage loss: 3.490
...80,000 Avarage loss: 3.472
...90,000 Avarage loss: 3.462
...100,000 Avarage loss: 3.449
...110,000 Avarage loss: 3.436
...120,000 Avarage loss: 3.423
...130,000 Avarage loss: 3.414
...140,000 Avarage loss: 3.401
...150,000 Avarage loss: 3.390
...160,000 Avarage loss: 3.383
...170,000 Avarage loss: 3.376
...180,000 Avarage loss: 3.369
...190,000 Avarage loss: 3.362
...200,000 Avarage loss: 3.351
...210,000 Avarage loss: 3.336
...220,000 Avarage loss: 3.332
...230,000 Avarage loss: 3.325
...240,000 Avarage loss: 3.322
...250,000 Avarage loss: 3.312
...260,000 Avarage loss: 3.307

Avarage loss: 3.306

each: every (0.658), any (0.554), all (0.441), several (0.371)
length: size (0.406), amount (0.394), cost (0.385), omphalos (0.339)
writer: author 

In [71]:
EMBEDDINGS_FILE = os.path.join(HOME_DIR, 'skip_gram.txt')
save_embeddings(EMBEDDINGS_FILE, skip_embeddings)

print('Embeddings file size: {:,d} bytes'.format(os.stat(EMBEDDINGS_FILE).st_size))

Embeddings file size: 54,399,384 bytes
